In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [13]:
years = [2017,2018,2019,2020,2021,2022]

In [14]:
categories = ['top-gc-riders', 'top-sprinters', 'top-time-trial-riders', 'best-classic-riders']

In [15]:
def chunks(lst, n, start):
    """Yield successive n-sized chunks from lst."""
    for i in range(start, len(lst), n):
        return lst[i:i + n]

In [16]:
def clean_scrape_profile(row, category, year):
    di = {}
    name = row[1].text.lstrip()
    team = row[2].text
    points = row[3].text
    ranking = row[4].text
    di['name'] = name
    di['team'] = team
    di['points'] = points
    di['ranking'] = ranking
    di['type'] = category
    di['year'] = year
    return di

In [17]:
def clean_scrape_performance(row, year):
    di = {}
    name_s = row[1].text.strip()
    name_ls = name_s.split()
    name = ' '.join([i.capitalize() for i in name_ls])
    gc_result = row[2].text
    giro_w = row[3].text
    stage_result = row[4].text
    stage_wins = row[5].text
    top10 = row[5].text
    total = row[7].text
    di['name'] = name
    di['gc_result'] = gc_result
    di['giro_w'] = giro_w
    di['stage_result'] = stage_result
    di['stage_wins'] = stage_wins
    di['top10'] = top10
    di['giro_total'] = total
    di['year'] = year
    return di

In [18]:
def clean_scrape_teams(row, year):
    di = {}
    team = row[1].text.lstrip()
    division = row[2].text
    points = row[3].text
    di['team'] = team
    di['team_class'] = division
    di['team_points'] = points
    di['year'] = year
    return di

In [19]:
def clean_age(row, year):
    di = {}
    name = row[1].text.lstrip()
    team = row[2].text
    age = row[3].text
    di['name'] = name
    di['team'] = team
    di['age'] = age[0:2]
    di['year'] = year
    return di

In [20]:
def clean_scrape_stages(row, year, stage, clas, last=False):
    di = {}
    rnk = row[0]
    if last==False:
        try:
            name = row[1].a.text.lstrip()
        except AttributeError:
            name = np.nan
    else:
        try:
            name = row[3].a.text.lstrip()
        except (IndexError, AttributeError):
            name = row[1].a.text.lstrip() 
    #di['key'] = name + '-' + str(year) + '-' + str(stage)
    di['name'] = name
    if len(clas) > 0:
        di['clas'] = clas
    di['clas'] = 'stage'
    di['rnk'] = rnk.text
    return di

In [21]:
def clean_scrape_stages(row, year, stage):
    di = {}
    rnk = row[0].text
    name = row[0].text
    di['rnk'] = rnk
    di['name'] = name
    di['key'] = str(year) + '-' + str(stage) + '-stage'

In [22]:
master_df = pd.DataFrame(columns=['name','team','points','ranking','type','year'])

In [23]:
all_riders = []

In [25]:
def get_age(race):
    years = [2017,2018,2019,2020,2021,2022]
    master_df_ls = []
    for year in years:
            url = f'https://www.procyclingstats.com/race/{race}/{year}/gc/startlist/youngest-competitors'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 4)
            ls = []
            for chunk in full_list_chunks:
                try:
                    di = clean_age(chunks(td, 4, chunk), year)
                    ls.append(di)
                except IndexError:
                    continue
                sub_df = pd.DataFrame.from_dict(ls)
                master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_age.csv', index=False)
for race in ['giro-d-italia', 'vuelta-a-espana', 'tour-de-france']:
#for race in [ 'tour-de-france']:

    get_age(race)

In [26]:
def get_teams(race):
    years = [2017,2018,2019,2020,2021]
    master_df_ls = []
    for year in years:
            url = f'https://www.procyclingstats.com/race/{race}/{year}/gc/startlist/teams-ranked'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 4)
            ls = []
            for chunk in full_list_chunks:
                try:
                    di = clean_scrape_teams(chunks(td, 4, chunk), year)
                    ls.append(di)
                except IndexError:
                    continue
                sub_df = pd.DataFrame.from_dict(ls)
                master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_teams.csv', index=False)

for race in ['giro-d-italia', 'vuelta-a-espana', 'tour-de-france']:

    get_teams(race)

In [1]:
def get_performance(race):
    years = [2017,2018,2019,2020,2021,2022]
    master_df_ls = []
    for year in years:
            url = f'https://www.procyclingstats.com/race/{race}/{year}/gc/startlist/previous-performance'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 9)
            ls = []
            for chunk in full_list_chunks:
                di = clean_scrape_performance(chunks(td, 9, chunk), year)
                ls.append(di)
            sub_df = pd.DataFrame.from_dict(ls)
            master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv(f'data_performance{race}.csv', index=False)


for race in ['giro-d-italia', 'vuelta-a-espana', 'tour-de-france']:

    get_performance(race)

NameError: name 'requests' is not defined

In [34]:
def get_profile(race):
    categories = ['top-gc-riders', 'top-sprinters', 'top-time-trial-riders', 'best-classic-riders']
    years = [2017,2018,2019,2020,2021,2022]
    master_df_ls = []
    for year in years:
        for category in categories:
            url = f'https://www.procyclingstats.com/race.php?missing=0&filter=Filter&id1={race}&id2={year}&id3=gc&id4=startlist&id5={category}'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            td = soup.find_all('td')
            full_list_chunks = np.arange(0,len(td), 5)
            ls = []
            for chunk in full_list_chunks:
                di = clean_scrape_profile(chunks(td, 5, chunk), category, year)
                ls.append(di)
            sub_df = pd.DataFrame.from_dict(ls)
            master_df_ls.append(sub_df)
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_profile.csv', index=False)
    
for race in ['giro-d-italia', 'vuelta-a-espana', 'tour-de-france']:
    get_profile(race)
        
        

In [35]:
def get_stages(race):
    years = [2017]#,2018,2019,2020,2021,2022]
    master_df_ls = []
    for year in years:
        #print(year)
        for stage in range(20,22):
                url = f'https://www.procyclingstats.com/race/{race}/{year}/stage-{stage}'
                print(url)
                #print(url)
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                td = soup.find_all('tr', class_=None)
                length = 4
                full_list_chunks = np.arange(0,len(td), length)
                print(full_list_chunks)
                ls = []
                for chunk in full_list_chunks:
                        di = clean_scrape_stages(chunks(td, length, chunk), year, stage)
                        ls.append(di)
                sub_df = pd.DataFrame.from_dict(ls)
                master_df_ls.append(sub_df)
                print(f"{year}-{stage}—done-norm")
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_stages.csv', index=False)
    
for race in ['giro-d-italia', 'vuelta-a-espana', 'tour-de-france']:

    get_stages(race)
    

https://www.procyclingstats.com/race/giro-d-italia/2017/stage-20
[  0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68
  72  76  80  84  88  92  96 100 104 108 112 116 120 124 128 132 136 140
 144 148 152 156 160 164 168 172 176 180 184 188 192 196 200 204 208 212
 216 220 224 228 232 236 240 244 248 252 256 260 264 268 272 276 280 284
 288 292 296 300 304 308 312 316 320 324 328 332 336 340 344 348 352 356
 360 364 368 372 376 380 384 388 392 396 400 404 408 412 416 420 424 428
 432 436 440 444 448 452 456 460 464 468 472 476 480 484 488 492 496 500
 504 508 512 516 520 524 528 532 536 540 544 548 552 556 560 564 568 572]
2017-20—done-norm
https://www.procyclingstats.com/race/giro-d-italia/2017/stage-21
[  0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68
  72  76  80  84  88  92  96 100 104 108 112 116 120 124 128 132 136 140
 144 148 152 156 160 164 168 172 176 180 184 188 192 196 200 204 208 212
 216 220 224 228 232 236 240 244 248 252 256 260

In [7]:
# stage scraper
stages = {}
for day in range(1,22):
    url = f'https://www.procyclingstats.com/race/giro-d-italia/2017/stage-{day}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser').find('tbody')
    tr = soup.find_all('tr', class_=None)
    dictionary = {}
    for i in range(len(tr)):
        td = tr[i].find_all('td', class_=None)[:2]
        name = td[1].find('a').text.split()
        name = ' '.join([x.capitalize() for x in name])
        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
        dictionary[name] = td[0].text
    stages[str(day)+'-stage'] = dictionary

In [8]:
stages

{'1-stage': {'Pöstlberger Lukas': '1',
  'Ewan Caleb': '2',
  'Greipel André': '3',
  'Nizzolo Giacomo': '4',
  'Modolo Sacha': '5',
  'Sbaragli Kristian': '6',
  'Stuyven Jasper': '7',
  'Gibbons Ryan': '8',
  'Bennett Sam': '9',
  'Bauhaus Phil': '10',
  'Mühlberger Gregor': '11',
  'Gaviria Fernando': '12',
  'Battaglin Enrico': '13',
  'Kuznetsov Viacheslav': '14',
  'Jungels Bob': '15',
  'Mareczko Jakub': '16',
  'Barbin Enrico': '17',
  'Thomas Geraint': '18',
  'Ponzi Simone': '19',
  'Costa Rui': '20',
  'Rojas José Joaquín': '21',
  'Sánchez Luis León': '22',
  'Nibali Vincenzo': '23',
  'Tsatevich Alexey': '24',
  'Quintana Nairo': '25',
  'Rosa Diego': '26',
  'Yates Adam': '27',
  'Dumoulin Tom': '28',
  'Ventoso Francisco José': '29',
  'Dennis Rohan': '30',
  'Pozzovivo Domenico': '31',
  'Preidler Georg': '32',
  'Rosskopf Joey': '33',
  'Selig Rüdiger': '34',
  'Polanc Jan': '35',
  'Konrad Patrick': '36',
  'Bennati Daniele': '37',
  'Ferrari Roberto': '38',
  'Van Ga

In [6]:

teams = {}
url = f'https://www.procyclingstats.com/race/giro-d-italia/2017/stage-1'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser').find('tbody')
tr = soup.find_all('tr', class_=None)
dictionary = {}
for i in range(len(tr)):
        td = tr[i].find_all('td', class_=None)[:2]
        name = td[1].find('a').text.split()
        name = ' '.join([x.capitalize() for x in name])
        team = td[1].find_all('span')[-1].text
        teams[name] = team
teams

{'Pöstlberger Lukas': 'BORA - hansgrohe',
 'Ewan Caleb': 'ORICA-Scott',
 'Greipel André': 'Lotto Soudal',
 'Nizzolo Giacomo': 'Trek - Segafredo',
 'Modolo Sacha': 'UAE Team Emirates',
 'Sbaragli Kristian': 'Team Dimension Data',
 'Stuyven Jasper': 'Trek - Segafredo',
 'Gibbons Ryan': 'Team Dimension Data',
 'Bennett Sam': 'BORA - hansgrohe',
 'Bauhaus Phil': 'Team Sunweb',
 'Mühlberger Gregor': 'BORA - hansgrohe',
 'Gaviria Fernando': 'Quick-Step Floors',
 'Battaglin Enrico': 'Team LottoNL-Jumbo',
 'Kuznetsov Viacheslav': 'Team Katusha Alpecin',
 'Jungels Bob': 'Quick-Step Floors',
 'Mareczko Jakub': 'Wilier Triestina - Selle Italia',
 'Barbin Enrico': 'Bardiani - CSF',
 'Thomas Geraint': 'Team Sky',
 'Ponzi Simone': 'CCC Sprandi Polkowice ',
 'Costa Rui': 'UAE Team Emirates',
 'Rojas José Joaquín': 'Movistar Team',
 'Sánchez Luis León': 'Astana Pro Team',
 'Nibali Vincenzo': 'Bahrain Merida Pro Cycling Team',
 'Tsatevich Alexey': 'Gazprom-RusVelo',
 'Quintana Nairo': 'Movistar Team',


In [272]:
#classification scraper
class_dictionary = {}
for day in range(1,22):
    if day < 21:
        for classification in [ '-gc', '-points', '-kom', '-youth']:
            url = f'https://www.procyclingstats.com/race/giro-d-italia/2017/stage-{day}{classification}'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser').find('tbody')
            tr = soup.find_all('tr', class_=None)
            dictionary = {}
            for i in range(len(tr)):
                td = tr[i].find_all('td', class_=None)[:2]
                name = td[1].find('a').text.split()
                name = ' '.join([x.capitalize() for x in name])
                #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                dictionary[name] = td[0].text
            class_dictionary[str(day)+classification] = dictionary
    else:
        for classification in [ 'gc', 'points', 'kom', 'youth']:
            url = f'https://www.procyclingstats.com/race/giro-d-italia/2017/{classification}'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser').find('tbody')
            tr = soup.find_all('tr', class_=None)
            dictionary = {}
            for i in range(len(tr)):
                td = tr[i].find_all('td', class_=None)[:2]
                name = td[1].find('a').text.split()
                name = ' '.join([x.capitalize() for x in name])
                #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                dictionary[name] = td[0].text
            classification[str(day)+'-'+classification] = dictionary
        

KeyboardInterrupt: 

In [349]:
#new classification scraper
class_dictionary = {}
for day in range(10,12):
        url = f'https://www.procyclingstats.com/race/giro-d-italia/2017/stage-{day}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser').find_all('table', class_='results basic moblist10')
        for i, j in enumerate([ '-stage', '-gc', '-points', '-youth', '-kom', '-teams']):
            if j !='-teams':
                tr = soup[i].find_all('tr')
                dictionary = {}
                for i in range(1,len(tr)):
                    td = tr[i].find_all('td', class_=None)[:2]
                    name = td[1].find('a').text.split()
                    name = ' '.join([x.capitalize() for x in name])
                        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                    dictionary[name] = td[0].text
                    class_dictionary[str(day)+j] = dictionary
            else:
                tr = soup[i].find_all('tr')
                dictionary = {}
                for i in range(1,len(tr)):
                    td = tr[i].find_all('td', class_=None)[:2]
                    name = td[1].find('a').text
                        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                    dictionary[name] = td[0].text
                    class_dictionary[str(day)+j] = dictionary

In [411]:
url = 'https://www.procyclingstats.com/race/giro-d-italia/2020/stage-12'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser').find('ul', class_='infolist')

In [416]:
chunks(soup.find_all('div'),2)

TypeError: object of type 'int' has no len()

In [51]:
for year in [2016]:
    for race in ['vuelta-a-espana']:
        for stage in [1,2,3]:

        
            teams = {}
            url = f'https://www.procyclingstats.com/race/{race}/{year}/stage-{stage}'
            print(url)
            if url in ['https://www.procyclingstats.com/race/vuelta-a-espana/2016/stage-1', 'https://www.procyclingstats.com/race/vuelta-a-espana/2016/stage-2']:
                print('y')

https://www.procyclingstats.com/race/vuelta-a-espana/2016/stage-1
y
https://www.procyclingstats.com/race/vuelta-a-espana/2016/stage-2
y
https://www.procyclingstats.com/race/vuelta-a-espana/2016/stage-3


In [46]:
if 'a' in ['a','b']:
    print('y')

y


In [62]:
import termcolor

ModuleNotFoundError: No module named 'termcolor'

In [65]:
!pip install termcolor

  Using cached termcolor-1.1.0-py3-none-any.whl
You should consider upgrading via the '/Users/michieldekoninck/.pyenv/versions/3.8.12/envs/boonen/bin/python3.8 -m pip install --upgrade pip' command.


In [9]:
# Scraper
for year in [2016, 2017, 2018, 2019, 2020, 2021,2022]:
    for race in ['giro-d-italia', 'tour-de-france', 'vuelta-a-espana']:
        for stage in range(1,22):
            try:
                url = f'https://www.procyclingstats.com/race/{race}/{year}/stage-{stage}'
                print(f"'y' - {url}")
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser').find('tbody')
                tr = soup.find_all('tr')
                dictionary = {}
                for i in range(len(tr)):
                        td = tr[i].find_all('td', class_=None)[:2]
                        name = td[1].find('a').text.split()
                        name = ' '.join([x.capitalize() for x in name])
                        team = td[1].find_all('span')[-1].text
                        teams[name] = team

                teams_dictionary = {}
                class_dictionary = {}
                # for day in range(1,22):
                #     url = f'https://www.procyclingstats.com/race/{race}/{year}/stage-{day}'
                #     print(url)
                soup = BeautifulSoup(response.content, 'html.parser').find_all('table', class_='results basic moblist10')
                if len(soup) == 6:
                        for k, m in enumerate([ '-stage', '-gc', '-points', '-youth', '-kom']):
                            print(k,m)
                            tr = soup[k].find_all('tr')
                            dictionary = {}
                            for i in range(1,len(tr)):
                                td = tr[i].find_all('td', class_=None)[:2]
                                name = td[1].find('a').text.split()
                                name = ' '.join([x.capitalize() for x in name])
                                        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                                dictionary[name] = td[0].text
                                class_dictionary[str(day)+m] = dictionary
                        #get teams
                        tr = soup[-1].find_all('tr')
                        dictionary = {}
                        for i in range(1,len(tr)):
                            td = tr[i].find_all('td', class_=None)[:2]
                            name = td[1].find('a').text
                                        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                            dictionary[name] = td[0].text
                            teams_dictionary[str(day)+'-teams'] = dictionary
                        
                else:
                        for k, m in enumerate([ '-stage', '-gc', '-points', '-youth']):
                            tr = soup[k].find_all('tr')
                            dictionary = {}
                            for i in range(1,len(tr)):
                                td = tr[i].find_all('td', class_=None)[:2]
                                name = td[1].find('a').text.split()
                                name = ' '.join([x.capitalize() for x in name])
                                        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                                dictionary[name] = td[0].text
                                class_dictionary[str(day)+m] = dictionary
                        
                        tr = soup[k].find_all('tr')
                        dictionary = {}
                        for i in range(1,len(tr)):
                            td = tr[i].find_all('td', class_=None)[:2]
                            name = td[1].find('a').text
                                        #name = ''.join(name.apply(lambda x: x.capitalize for x in name))
                            dictionary[name] = td[0].text
                            teams_dictionary[str(day)+'-teams'] = dictionary
            except IndexError:
                print(f"'NOOO' - {url}")
                            
            #merge frames + csv
            teams_rider = pd.DataFrame.from_dict(teams, orient='index', columns=['team'])
            all_df = pd.DataFrame.from_dict(class_dictionary, orient='index').T
            teams_df = pd.DataFrame.from_dict(teams_dictionary, orient='index').T.reset_index().rename(columns={'index':'team'})
            combo = all_df.join(teams_rider)
            combo = combo.reset_index().merge(teams_df, on='team', how='left')
            combo['year'] = str(year)
            combo.to_csv(f'data_raw_{year}_{race}.csv')

'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-1
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-2
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-3
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-4
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-5
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-6
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-7
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-8
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race/giro-d-italia/2016/stage-9
0 -stage
1 -gc
2 -points
3 -youth
4 -kom
'y' - https://www.procyclingstats.com/race

KeyboardInterrupt: 

In [407]:
zestien = pd.read_csv('data_raw_2021.csv').rename(columns={'index': 'rider'})
zestien

,Unnamed: 0,rider,1-stage,1-gc,1-points,1-youth,2-stage,2-gc,2-points,2-kom,...,13-teams,14-teams,15-teams,16-teams,17-teams,18-teams,19-teams,20-teams,21-teams,year
0,0,Ganna Filippo,1,1,1.0,1.0,20,1.0,6.0,1.0,...,1,1,2,2.0,1.0,2.0,1.0,1.0,1.0,2021
1,1,Affini Edoardo,2,2,2.0,2.0,101,2.0,8.0,2.0,...,7,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021
2,2,Foss Tobias,3,3,3.0,3.0,100,3.0,11.0,3.0,...,7,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021
3,3,Almeida João,4,4,4.0,4.0,50,5.0,14.0,5.0,...,6,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021
4,4,Cavagna Rémi,5,5,5.0,NaN,79,6.0,16.0,NaN,...,6,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021
5,5,Van Emden Jos,6,6,6.0,NaN,60,7.0,19.0,NaN,...,7,5,7,4.0,2.0,5.0,3.0,2.0,2.0,2021
6,6,Evenepoel Remco,7,7,7.0,5.0,104,4.0,21.0,4.0,...,6,7,8,10.0,10.0,9.0,9.0,8.0,8.0,2021
7,7,Walscheid Max,8,8,8.0,NaN,82,8.0,24.0,NaN,...,22,22,22,22.0,22.0,22.0,22.0,21.0,21.0,2021
8,8,Brändle Matthias,9,9,9.0,NaN,113,9.0,26.0,NaN,...,17,16,16,17.0,17.0,16.0,17.0,17.0,17.0,2021
9,9,Moscon Gianni,10,10,10.0,NaN,55,10.0,30.0,NaN,...,1,1,2,2.0,1.0,2.0,1.0,1.0,1.0,2021


In [304]:
pd.set_option('display.max_rows', None)
zestien[zestien['rider']=='Roglič Primož'].T

,1
rider,Roglič Primož
1-stage,2
2-stage,61
3-stage,174
4-stage,193
5-stage,142
6-stage,61
7-stage,159
8-stage,162
9-stage,1


def get_stages():
    years = [2017]#,2018,2019,2020,2021]
    master_df_ls = []
    for year in years:
        print(year)
        for stage in range(1,22):
            print(stage)
            for clas in ['']:#, '-gc', '-points', '-kom', '-youth', '-teams']:
                print(clas)
                #try:
                url = f'https://www.procyclingstats.com/race/giro-d-italia/{year}/stage-{stage}{clas}'
                print(url)
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                td = soup.find_all('td', class_=None)
                dictionary = {
                                '':4,
                                '-gc':3,
                                '-points':4,
                                '-kom':4,
                                '-youth':2,
                                '-teams':3
                            }
                length = dictionary[clas]
                full_list_chunks = np.arange(0,len(td), length)
                ls = []
                for chunk in full_list_chunks:
                        di = clean_scrape_stages(chunks(td, length, chunk), year, stage, clas)
                        ls.append(di)
                sub_df = pd.DataFrame.from_dict(ls)
                master_df_ls.append(sub_df)
                print(f"{year}-{stage}-{clas}—done-norm")
                # except IndexError:
                #         if clas != '':
                #             url = f"https://www.procyclingstats.com/race/giro-d-italia/{year}{clas.replace('-','/')}"
                #             print(url)
                #             response = requests.get(url)
                #             soup = BeautifulSoup(response.content, 'html.parser')
                #             td = soup.find_all('td', class_=None)
                #             dictionary = {
                #                 '':4,
                #                 '-gc':3,
                #                 '-points':4,
                #                 '-kom':4,
                #                 '-youth':2,
                #                 '-teams':3
                #             }
                #             length = dictionary[clas]
                #             full_list_chunks = np.arange(0,len(td), length)
                #             ls = []
                #             for chunk in full_list_chunks:
                #                 di = clean_scrape_stages(chunks(td, length, chunk), year, stage, clas.replace('/','-'), last=True)
                #                 ls.append(di)
                #             sub_df = pd.DataFrame.from_dict(ls)
                #             master_df_ls.append(sub_df)
                #             print(f"{year}-{stage}-{clas}—done-spec")
                #         else:
                #             pass
    X1 = pd.concat(master_df_ls, ignore_index=True)
    X1.to_csv('data_stages.csv', index=False)
get_stages()

In [201]:
np.arange(1,22,1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21])

In [83]:
profile = pd.read_csv('data_profile.csv').drop_duplicates()
age = pd.read_csv('data_age.csv').drop_duplicates()
performance = pd.read_csv('data_performance.csv').drop_duplicates()
teams = pd.read_csv('data_teams.csv').drop_duplicates()

In [84]:
age

,name,team,age,year
0,Albanese Vincenzo,Bardiani - CSF,20,2017
2,Martínez Daniel Felipe,Wilier Triestina - Selle Italia,21,2017
5,Pedersen Mads,Trek - Segafredo,21,2017
9,De Plus Laurens,Quick-Step Floors,21,2017
14,Schlegel Michal,CCC Sprandi Polkowice,21,2017
...,...,...,...,...
82520,Richeze Maximiliano,UAE Team Emirates,38,2021
82701,Visconti Giovanni,Bardiani-CSF-Faizanè,38,2021
82883,Keisse Iljo,Deceuninck - Quick Step,38,2021
83066,Pozzovivo Domenico,Team Qhubeka ASSOS,38,2021


In [94]:
table2 = pd.merge(profile, age,  how='outer', left_on=['name','year', 'team'], right_on = ['name','year', 'team'])

In [157]:
'-'.replace('-','/')

'/'

In [95]:
#table3 = pd.merge(table2, performance, how='outer', left_on=['name','year'], right_on = ['name','year'])

In [134]:
table4 = pd.merge(table2, teams, how='outer', left_on=['team','year'], right_on = ['team','year']).drop(columns=['ranking', 'team_class'])

In [144]:
table4['year'] = table4['year'].astype('str')
table4['key'] = table4[['name', 'year']].agg('-'.join, axis=1)
table4

,name,team,points,type,year,age,team_points,key
0,Quintana Nairo,Movistar Team,1550.0,top-gc-riders,2017,27,3999,Quintana Nairo-2017
1,Quintana Nairo,Movistar Team,34.6,top-sprinters,2017,27,3999,Quintana Nairo-2017
2,Quintana Nairo,Movistar Team,358.0,top-time-trial-riders,2017,27,3999,Quintana Nairo-2017
3,Izagirre Gorka,Movistar Team,267.0,top-gc-riders,2017,29,3999,Izagirre Gorka-2017
4,Izagirre Gorka,Movistar Team,39.3,top-sprinters,2017,29,3999,Izagirre Gorka-2017
...,...,...,...,...,...,...,...,...
2159,Bevin Patrick,Israel Start-Up Nation,8.0,top-sprinters,2021,30,1405,Bevin Patrick-2021
2160,Bevin Patrick,Israel Start-Up Nation,85.0,top-time-trial-riders,2021,30,1405,Bevin Patrick-2021
2161,Martin Dan,Israel Start-Up Nation,66.8,top-sprinters,2021,34,1405,Martin Dan-2021
2162,Martin Dan,Israel Start-Up Nation,38.0,top-time-trial-riders,2021,34,1405,Martin Dan-2021


In [145]:
table5 = table4.pivot_table(values='points', index='key', columns='type', aggfunc='first').reset_index()
table5

type,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders
0,Ackermann Pascal-2019,748.0,28.0,1069.4,NaN
1,Affini Edoardo-2020,40.0,100.0,55.0,273.0
2,Affini Edoardo-2021,3.0,5.0,NaN,109.0
3,Albanese Vincenzo-2017,90.0,NaN,246.0,30.0
4,Albanese Vincenzo-2021,36.0,NaN,34.0,NaN
...,...,...,...,...,...
845,van den Berg Lars-2021,NaN,NaN,26.0,NaN
846,van der Hoorn Taco-2021,32.0,15.0,NaN,NaN
847,Černý Josef-2019,15.0,149.0,126.2,188.0
848,Černý Josef-2020,39.0,23.0,NaN,162.0


In [150]:
table6 = pd.merge(table4, table5, how='outer', left_on='key', right_on = 'key').drop(columns=['type', 'points']).drop_duplicates()
table6

,name,team,year,age,team_points,key,best-classic-riders,top-gc-riders,top-sprinters,top-time-trial-riders
0,Quintana Nairo,Movistar Team,2017,27,3999,Quintana Nairo-2017,NaN,1550.0,34.6,358.0
3,Izagirre Gorka,Movistar Team,2017,29,3999,Izagirre Gorka-2017,76.0,267.0,39.3,172.0
7,Amador Andrey,Movistar Team,2017,30,3999,Amador Andrey-2017,37.0,180.0,85.0,202.0
11,de la Parte Víctor,Movistar Team,2017,30,3999,de la Parte Víctor-2017,29.0,145.0,NaN,24.0
14,Anacona Winner,Movistar Team,2017,28,3999,Anacona Winner-2017,16.0,123.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2150,Niv Guy,Israel Start-Up Nation,2021,27,1405,Niv Guy-2021,NaN,30.0,NaN,25.0
2152,De Marchi Alessandro,Israel Start-Up Nation,2021,34,1405,De Marchi Alessandro-2021,110.0,25.0,NaN,47.0
2155,Cimolai Davide,Israel Start-Up Nation,2021,31,1405,Cimolai Davide-2021,19.0,20.0,146.0,NaN
2158,Bevin Patrick,Israel Start-Up Nation,2021,30,1405,Bevin Patrick-2021,NaN,9.0,8.0,85.0


In [ ]:
td = soup.find_all('td')

750

In [ ]:
int(len(td)/5)

150

In [ ]:
full_list_chunks = np.arange(0,len(td), 5)


In [ ]:
def chunks(lst, n, start):
    """Yield successive n-sized chunks from lst."""
    for i in range(start, len(lst), n):
        return lst[i:i + n]

In [ ]:
def clean_scrape(row):
    di = {}
    name = row[1].text.lstrip()
    team = row[2].text
    points = row[3].text
    ranking = row[4].text
    di['name'] = name
    di['team'] = team
    di['points'] = points
    di['ranking'] = ranking
    di['type'] = categories
    di['year'] = '2021'
    return di

In [ ]:
ls = []
for chunk in full_list_chunks:
    di = clean_scrape(chunks(td, 5, chunk))
    ls.append(di)

In [ ]:
pd.DataFrame.from_dict(ls)

,name,team,points,ranking,type,year
0,Evenepoel Remco,Deceuninck - Quick Step,700,4,top-gc-riders,2021
1,Ulissi Diego,UAE Team Emirates,566,8,top-gc-riders,2021
2,Landa Mikel,Bahrain - Victorious,528,11,top-gc-riders,2021
3,Yates Simon,Team BikeExchange,528,10,top-gc-riders,2021
4,Hindley Jai,Team DSM,516,14,top-gc-riders,2021
...,...,...,...,...,...,...
145,Covi Alessandro,UAE Team Emirates,5,846,top-gc-riders,2021
146,Duchesne Antoine,Groupama - FDJ,5,885,top-gc-riders,2021
147,Petilli Simone,Intermarché - Wanty - Gobert Matériaux,5,871,top-gc-riders,2021
148,Nizzolo Giacomo,Team Qhubeka ASSOS,5,822,top-gc-riders,2021


In [ ]:
#row = chunks(td, 5, 0)
row = chunks(td,5,5)
row

[<td>2</td>,
 <td><span class="flag it"></span> <a href="rider/diego-ulissi"><span class="uppercase">Ulissi</span> Diego</a></td>,
 <td>UAE Team Emirates</td>,
 <td>566</td>,
 <td>8</td>]

In [ ]:
clean_scrape(row)

{' Ulissi Diego': ['UAE Team Emirates', '566', '8']}

In [ ]:
di = {}
#di2 = {}
def clean_scrape(row):
    name = row[1].text
    team = row[2].text
    points = row[3].text
    ranking = row[4].text
    di[name] = [team, points, ranking]
    return di

SyntaxError: 'return' outside function (1095910690.py, line 9)

In [ ]:
[di,di2]

[{}, {' Evenepoel Remco': ['Deceuninck - Quick Step', '700', '4']}]

In [ ]:
pd.DataFrame.from_dict(di).T

,0,1,2
Evenepoel Remco,Deceuninck - Quick Step,700,4
